In [41]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.eval_measures import rmse
import optuna

In [42]:
df = pd.read_csv('../../train_belgrade.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [43]:
train_size = int(len(df) * 0.8)
train, test = df[0:train_size], df[train_size:]

In [44]:
target_column = 'Detections'

print(train.columns)


Index(['Unnamed: 0', 'Timestamp', 'Detections', 'index', 'WeatherMain',
       'Temperature', 'WeatherDescription'],
      dtype='object')


In [45]:
def objective(trial):
    params = {
        'trend': trial.suggest_categorical('trend', [None, 'add']),
        'seasonal': trial.suggest_categorical('seasonal', [None, 'add']),
        'seasonal_periods': trial.suggest_categorical('seasonal_periods', [120, 288, 288*7])
    }

    model = ExponentialSmoothing(train[target_column], trend=params['trend'], seasonal=params['seasonal'], seasonal_periods=params['seasonal_periods'])
    fitted_model = model.fit(optimized=True)

    predictions = fitted_model.forecast(len(test))

    mse = mean_squared_error(test[target_column], predictions)

    return mse


In [46]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)


[I 2024-02-18 21:58:05,854] A new study created in memory with name: no-name-2191aca9-30b3-4a1c-ba17-35ffbb2f6bc3
[I 2024-02-18 21:58:06,800] Trial 0 finished with value: 145.13829456430744 and parameters: {'trend': None, 'seasonal': 'add', 'seasonal_periods': 120}. Best is trial 0 with value: 145.13829456430744.
[I 2024-02-18 21:59:09,002] Trial 1 finished with value: 263.1092713299548 and parameters: {'trend': None, 'seasonal': 'add', 'seasonal_periods': 2016}. Best is trial 0 with value: 145.13829456430744.
[I 2024-02-18 21:59:09,638] Trial 2 finished with value: 160.9383405537474 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 2016}. Best is trial 0 with value: 145.13829456430744.
[I 2024-02-18 21:59:10,166] Trial 3 finished with value: 160.9383405537474 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 288}. Best is trial 0 with value: 145.13829456430744.
[I 2024-02-18 22:00:09,990] Trial 4 finished with value: 263.1092713299548 and para

In [47]:
best_params = study.best_params
best_model = ExponentialSmoothing(train[target_column], trend=best_params['trend'], seasonal=best_params['seasonal'], seasonal_periods=best_params['seasonal_periods']).fit()
best_predictions = best_model.forecast(len(test))

best_mse = mean_squared_error(test[target_column], best_predictions)

print(f"Best MSE: {best_mse}")


Best MSE: 145.13829456430744
